# **1 - Tarefa e Dados**
Descreva a tarefa escolhida e os dados. Escreva código que leia os dados e calcule e imprima quantas instâncias os dados têm. Também, seu código deve calcular a média de tokens por instância, isto é, quantos tokens, na média cada documento do seu conjunto de dados possui. Imprima esse único número.


A corpus escolhido foi o UTL que é um corpus com críticas de filmes e apps coletadas automaticamente de sites. As classes são: positiva ou negativa. Assim o usuário pode ter gostado ou não gostado do produto. Referência: https://github.com/RogerFig/UTLCorpus 


In [ ]:
# montando o drive do google onde o corpus se encontra
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# conferir que o Google Drive está devidamente montado (deve aparecer a pasta 'drive')
%ls -la

total 20
drwxr-xr-x 1 root root 4096 Nov 18 21:16 ./
drwxr-xr-x 1 root root 4096 Nov 18 21:13 ../
drwxr-xr-x 1 root root 4096 Nov 13 17:34 .config/
drwx------ 5 root root 4096 Nov 18 21:16 drive/
drwxr-xr-x 1 root root 4096 Nov 13 17:33 sample_data/


In [ ]:
# Entrando no diretório
%cd drive/My Drive/Colab Notebooks/NLP/trabalho

/content/drive/My Drive/Colab Notebooks/NLP/trabalho


In [ ]:
import pandas as pd

# abrindo o arquivo.
db.pd_read

In [ ]:
import nltk
#nltk.download('http://143.107.183.175:23780/UTLCorpus/UTLCorpus.csv')

#nltk.data('http://143.107.183.175:23780/UTLCorpus/UTLCorpus.csv')
nltk.data('dados/UTLCorpus.csv')

TypeError: ignored

# **2 - Visualização dos dados**
Coloque nesta seção os gráficos do PCA e do t-SNE, para cada representação. 

Responda também às seguintes perguntas: 

a) Existe algum padrão com relação às classes? 

b) Caso exista algum padrão, você pode concluir alguma coisa? 

c) Caso não exista, você consegue dizer se isso tem a ver com alguma representação ou classe?


## Divisão dos dados em treino e teste

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

## Representação TF-IDF
Você pode usar tanto o gensim quanto o scikit para montar esta representação, mas lembre-se que é importante fazer o pré-processamento dos textos.


## Representação com o word2vec
O modelo poderá ser o apresentado na aula 03 ou algum outro modelo pré-treinado como os existentes no repositório http://nilc.icmc.usp.br/nilc/index.php/repositorio-de-word-embeddings-do-nilc . Neste caso, cada documento deverá ser representado pelo vetor que resultar da média dos vetores de todas as palavras que o compõem. Em outras palavras, se D é composto pelas palavras w1, w2, …, wn, e seus vetores embeddings são v1, v2, …, vn, então a representação do documento de D será v = (v1 + v2 + … + vn) / n. 


## Extração de features do texto
Você deve pensar em ao menos 10 features para extrair do documento e que o possam representar. Aqui vão algumas sugestões: número de palavras, número de verbos, número de conjunções, número de palavras negativas, número de palavras fora do vocabulário, quantidades de entidades do tipo PESSOA, quantidade de entidades do tipo LOCAL, etc.


## Visualização dos seus dados com PCA

## Visualização dos seus dados com t-SNE

# **3 - Classificadores**
Descreva sucintamente os dois classificadores escolhidos (k-nn, SVM,RandomForest...). Você usou algum parâmetro que não seja padrão? Se sim, mencione nesta seção. 


# **4 - Resultados**
Escreva código que execute a validação cruzada em 5-folds para os dois classificadores escolhidos. Também responda às seguintes perguntas: Os embeddings realmente mostraram um resultado melhor que o TF-IDF? Se não, qual foi a representação que teve o melhor desempenho? A diferença foi muito grande?

# **5 - Conclusão**
Por fim fale aqui o que você conclui das visualizações e dos resultados. Tente explicar em detalhes por que um resultado, na sua opinião, foi melhor do que outro. Esta explicação pode incluir hipóteses para resultados melhores ou resultados piores. Também pode falar das dificuldades enfrentadas durante o trabalho e como conseguiu contorná-las.
